# Ejercicio árboles 

En este ejercicio construiremos algunos árboles de clasificación y veremos cómo seleccionar hiperparámetros para afinar medidas de desempeño.

In [ ]:
%autosave 0
import pandas as pd
import numpy as np
from plotnine import *
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

### Datos

Usaremos los datos de ventas de seguros de *Caravan*:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
caravan = pd.read_csv('../datos/caravan-insurance-challenge.csv')
columnas = caravan.columns[2:86]
print(columnas)
def preprocesar_caravan(datos, tipo, columnas):
    # filtrar tipo
    datos_p = datos[datos["ORIGIN"] == tipo].copy()
    # variable respuesta
    y = datos_p["CARAVAN"].values
    datos_p = datos_p[columnas]
    datos_tipo = pd.get_dummies(datos_p.MOSHOOFD, prefix="MOSHOODFD_", drop_first = True)
    datos_p = datos_p.drop(columns = ["MOSHOOFD"])
    datos_p = pd.concat([datos_tipo, datos_p], axis = 1, sort=False)
    columnas_x = datos_p.columns
    #datos_origen = datos[datos["ORIGIN"] == tipo].drop(columns = ["ORIGIN"])
    X = datos_p.values
    return X, y, columnas_x
X_ent, y_ent, columnas_x = preprocesar_caravan(caravan, "train", columnas)
print(X_ent.shape)
print(columnas_x)
np.unique(y_ent, return_counts=True)


MOSTYPE: Customer Subtype; see L0 MAANTHUI: Number of houses 1 - 10 MGEMOMV: Avg size household 1 - 6 MGEMLEEF: Avg age; see L1 MOSHOOFD: Customer main type; see L2

MGODRK: Roman catholic MGODPR: Protestant … MGODOV: Other religion MGODGE: No religion MRELGE: Married MRELSA: Living together MRELOV: Other relation MFALLEEN: Singles MFGEKIND: Household without children MFWEKIND: Household with children MOPLHOOG: High level education MOPLMIDD: Medium level education MOPLLAAG: Lower level education MBERHOOG: High status MBERZELF: Entrepreneur MBERBOER: Farmer MBERMIDD: Middle management MBERARBG: Skilled labourers MBERARBO: Unskilled labourers MSKA: Social class A MSKB1: Social class B1 MSKB2: Social class B2 MSKC: Social class C MSKD: Social class D MHHUUR: Rented house MHKOOP: Home owners MAUT1: 1 car MAUT2: 2 cars MAUT0: No car MZFONDS: National Health Service MZPART: Private health insurance MINKM30: Income < 30.000 MINK3045: Income 30-45.000 MINK4575: Income 45-75.000 MINK7512: Income 75-122.000 MINK123M: Income >123.000 MINKGEM: Average income MKOOPKLA: Purchasing power class

PWAPART: Contribution private third party insurance PWABEDR: Contribution third party insurance (firms) … PWALAND: Contribution third party insurane (agriculture) PPERSAUT: Contribution car policies PBESAUT: Contribution delivery van policies PMOTSCO: Contribution motorcycle/scooter policies PVRAAUT: Contribution lorry policies PAANHANG: Contribution trailer policies PTRACTOR: Contribution tractor policies PWERKT: Contribution agricultural machines policies PBROM: Contribution moped policies PLEVEN: Contribution life insurances PPERSONG: Contribution private accident insurance policies PGEZONG: Contribution family accidents insurance policies PWAOREG: Contribution disability insurance policies PBRAND: Contribution fire policies PZEILPL: Contribution surfboard policies PPLEZIER: Contribution boat policies PFIETS: Contribution bicycle policies PINBOED: Contribution property insurance policies PBYSTAND: Contribution social security insurance policies AWAPART: Number of private third party insurance 1 - 12 AWABEDR: Number of third party insurance (firms) … AWALAND: Number of third party insurance (agriculture) APERSAUT: Number of car policies ABESAUT: Number of delivery van policies AMOTSCO: Number of motorcycle/scooter policies AVRAAUT: Number of lorry policies AAANHANG: Number of trailer policies ATRACTOR: Number of tractor policies AWERKT: Number of agricultural machines policies ABROM: Number of moped policies ALEVEN: Number of life insurances APERSONG: Number of private accident insurance policies AGEZONG: Number of family accidents insurance policies AWAOREG: Number of disability insurance policies ABRAND: Number of fire policies AZEILPL: Number of surfboard policies APLEZIER: Number of boat policies AFIETS: Number of bicycle policies AINBOED: Number of property insurance policies ABYSTAND: Number of social security insurance policies CARAVAN: Number of mobile home policies 0 - 1

### Ajustando un aŕbol de clasificación

Ajusta un árbol relativamente chico. Podemos controlar con mínimo número de muestras por nodo, mínimo para por nodo para considerar dividirlo, máxima profundidad y mínimo decrecimiento de impureza para decidir cortar, 
por ejemplo:

In [ ]:
from sklearn import tree
arbol = tree.DecisionTreeClassifier(min_samples_split = 200,
                                  min_samples_leaf = 100, 
                                  max_depth = 4,
                                  criterion = "entropy",
                                  min_impurity_decrease = 0.002)

In [ ]:
arbol_caravan_ajuste = arbol.fit(X_ent, y_ent)

In [ ]:
# Graficar árbol
variables_nombres = columnas
fig, ax = plt.subplots(1, 1, figsize = (4,4), dpi = 300)
ax.set_title('Árbol completo')
anotacion = tree.plot_tree(arbol_caravan_ajuste, ax = ax,
                           feature_names = columnas_x,
                           filled = True, 
                           proportion = True)

In [ ]:
arbol = tree.DecisionTreeClassifier(min_samples_split = 100,
                                  min_samples_leaf = 50, 
                                  max_depth = 5)
arbol_caravan_ajuste = arbol.fit(X_ent, y_ent)

### Evaluación y comparación con regresión logística


Comparamos con regresión logística

In [ ]:
escalador = StandardScaler()
escalador_ajustado = escalador.fit(X_ent)
X_ent_esc = escalador_ajustado.transform(X_ent)
reg_caravan = LogisticRegression(solver='newton-cg', penalty="none")
reg_caravan_ajuste = reg_caravan.fit(X_ent_esc, y_ent)


In [ ]:
X_pr, y_pr, _ = preprocesar_caravan(caravan, "test", columnas)
X_pr_esc = escalador_ajustado.transform(X_pr)
# calcular probabilidades
probas_reg  = reg_caravan_ajuste.predict_proba(X_pr_esc)
probas_arbol = arbol_caravan_ajuste.predict_proba(X_pr)


Evaluamos los dos modelos en muestra de prueba. Primero el modelo de regresión logística:

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
tfp, tvp, cortes = roc_curve(y_pr, probas_reg[:,1])
datos_roc = pd.DataFrame({"tfp":tfp, "tvp":tvp, "corte":cortes})
datos_roc["tipo"] = "reg logistica" 
print("AUC prueba reg:", roc_auc_score(y_pr, probas_reg[:,1]).round(3))

Y ahora el árbol que ajustamos:

In [ ]:
tfp, tvp, cortes = roc_curve(y_pr, probas_arbol[:,1])
datos_roc_2 = pd.DataFrame({"tfp":tfp, "tvp":tvp, "corte":cortes})
datos_roc_2["tipo"] = "arbol_chico"
print("AUC prueba árbol:", roc_auc_score(y_pr, probas_arbol[:,1]).round(3))

In [ ]:
datos_roc = pd.concat([datos_roc, datos_roc_2])
(ggplot(datos_roc, aes("tfp", "tvp", group="tipo", colour="tipo")) 
  + geom_step(size=1.5)
  + geom_abline(slope=1, intercept=0)
  + xlab("Tasa de falsos positivos") + ylab("Sensibilidad")
  + labs(title ="Curva ROC prueba"))

Aunque en entrenamiento la situación se ve diferente:

In [ ]:
# predicciones
probas_reg  = reg_caravan_ajuste.predict_proba(X_ent_esc)
probas_arbol = arbol_caravan_ajuste.predict_proba(X_ent)
# reg
tfp, tvp, cortes = roc_curve(y_ent, probas_reg[:,1])
datos_roc = pd.DataFrame({"tfp":tfp, "tvp":tvp, "corte":cortes})
datos_roc["tipo"] = "reg logistica" 
# arbol
tfp, tvp, cortes = roc_curve(y_ent, probas_arbol[:,1])
# grafica
print("AUC entrena árbol:", roc_auc_score(y_ent, probas_arbol[:,1]).round(3))
print("AUC entrena regresión:", roc_auc_score(y_ent, probas_reg[:,1]).round(3))

datos_roc_2 = pd.DataFrame({"tfp":tfp, "tvp":tvp, "corte":cortes})
datos_roc_2["tipo"] = "arbol_chico"
datos_roc = pd.concat([datos_roc, datos_roc_2])
(ggplot(datos_roc, aes("tfp", "tvp", group="tipo", colour="tipo")) 
  + geom_step(size=1.5)
  + geom_abline(slope=1, intercept=0)
  + xlab("Tasa de falsos positivos") + ylab("Sensibilidad")
  + labs(title = "Curva ROC entrenamiento")
)

**Pregunta**: ¿cuál es la razón de este comportamiento malo del árbol en la muestra de prueba?

## Afinación de parámetros

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, \
    auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, \
    average_precision_score

hiperparams_rejilla = {
    'min_samples_split': [10, 50, 100], 
    'min_samples_leaf': [5, 10, 50],
    'max_depth' : [2, 4, 6, 8, 10],
    'min_impurity_decrease' : [1e-8, 1e-6, 1e-4]
}

calificadores = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'auc': make_scorer(roc_auc_score, needs_proba=True),
    'precision_promedio': make_scorer(average_precision_score, needs_proba = True)
}


arbol = tree.DecisionTreeClassifier()
cortes_val = StratifiedKFold(n_splits=10, random_state=2334)
# busqueda
busqueda_grid = GridSearchCV(arbol, hiperparams_rejilla, 
                            scoring=calificadores, 
                            refit="auc", 
                            return_train_score = True,
                            cv=cortes_val, n_jobs = -1)
busqueda_grid = busqueda_grid.fit(X_ent, y_ent)
print('Mejores parámetros para {}'.format("auc"))
print(busqueda_grid.best_params_)

In [ ]:
# Mostramos resultados
resultados = pd.DataFrame(busqueda_grid.cv_results_)
resultados
resultados = resultados.sort_values(by='mean_test_auc', ascending=False)
resultados.columns


In [ ]:
resultados[["param_max_depth", "param_min_impurity_decrease", "param_min_samples_leaf", 
        "param_min_samples_split", "params", "mean_test_auc", "std_test_auc", "mean_train_auc"]].round(3).head()

In [ ]:
probas_arbol = busqueda_grid.predict_proba(X_pr)
print("AUC:", roc_auc_score(y_pr, probas_arbol[:,1]).round(2))

Podemos afinar también la regularización en regresión logística

In [ ]:
hiperparams_rejilla = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0]
}

calificadores = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'auc': make_scorer(roc_auc_score, needs_proba=True),
    'precision_promedio': make_scorer(average_precision_score, needs_proba = True)
}


reg_log = LogisticRegression(solver = "newton-cg")
# busqueda
busqueda_grid = GridSearchCV(reg_log, hiperparams_rejilla, 
                               scoring=calificadores, 
                               refit="auc", 
                               cv=cortes_val, n_jobs = -1)
busqueda_grid = busqueda_grid.fit(X_ent_esc, y_ent)
print('Mejores parámetros para {}'.format("auc"))
print(busqueda_grid.best_params_)

In [ ]:
probas_reg = busqueda_grid.predict_proba(X_pr_esc)
print("AUC:", roc_auc_score(y_pr, probas_reg[:,1]).round(2))